In [1]:
%matplotlib inline
import copy
import qutip
from qutip import Qobj
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

from stateobj import Physics
import utilities as use
import circuits as qc

import os

from IPython.display import Markdown, display

In [2]:
TIMESTEPS = 5000
TIMEDELTA = 1e-01
OMEGA = 0.5  # Strength of Interaction

D = 5

p = Physics(dimension=D, interaction_strength=OMEGA, interaction_time=TIMEDELTA)

th = OMEGA * TIMEDELTA
alpha = complex(1/math.sqrt(2), 0)
beta = cmath.sqrt(1 - alpha**2)
phi = np.pi/2
# Phase shifts
delta1 = 0
delta2 = -phi

In [3]:
eta = use.create_ancilla_qobj(alpha, beta, phi)
rho1 = use.create_system_qobj('fock', n_dims=D)
rho2 = use.create_system_qobj('fock', n_dims=D)

display(qutip.tensor(rho1, rho2))

Quantum object: dims = [[5, 5], [5, 5]], shape = (25, 25), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0.

In [4]:
# Ancilla dephasing gate
class DephasingGate(qc.Gate):
    # Unitary evolution Matrix
    U = Qobj([[1, 0, 0], [0, 1, 0], [0, 0, cmath.exp(-1j*delta2)]])
    
    def evolve(self, system):
        return self.U * system * self.U.dag()
    
class MeasureGate(qc.Gate):
    def evolve(self, system):
        display(Markdown(f"{self.name.upper()} measure"))
        display(system)
        return system
    
class FirstTimeEvolutionGate(qc.Gate):
    # This acts only on the first system
    def evolve(self, system):
        U = (1j * th * p.V1).expm()
        print(U.shape)
        print(U.dims)
        print(system.dims)
        return U * system * U.dag()
    
class SecondTimeEvolutionGate(qc.Gate):
    def evolve(self, system):
        # This acts only on the second system
        U = (1j * th * p.V2).expm()
        print(U.shape)
        print(U.dims)
        print(system.dims)
        return U * system * U.dag()

    
class MasterEquationGate(qc.Gate):
    
    def dissipator(self, X: Qobj, system: Qobj):
            sandwich = X * system * X.dag()
            comm = qutip.commutator(X.dag()*X, system, kind='anti')
            return sandwich - 1/2 * comm
        
    def master_equation(self, system):
        ga = 2*alpha**2
        gb = (beta)**2*(1-np.cos(phi))
        # Bosonic Operators
        C = p.C
        Cp = p.Cp
        S = p.S
        Sd = p.S.dag()
        first_line = 0.5*self.dissipator(qutip.tensor(C, C) - 2*qutip.tensor(S, Sd), system) 
        first_line += self.dissipator(qutip.tensor(C, S) + qutip.tensor(S, Cp), system)
        second_line = 0.5*self.dissipator(qutip.tensor(Cp, Cp) - 2*qutip.tensor(Sd, S), system)
        second_line += self.dissipator(qutip.tensor(Cp, Sd) + qutip.tensor(Sd, C), system)
        return ga * first_line + gb * second_line
    
    def evolve(self, system):
        system_variation = self.master_equation(system)
        return system + system_variation
        


In [5]:
qutip.tensor(rho1, Qobj(1))

Quantum object: dims = [[5, 1], [5, 1]], shape = (5, 5), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

In [6]:
circuit = qc.Circuit(n_lanes=3)
circuit.populate_lane(0, -1, 'c1')
circuit.populate_lane(1, eta, 'eta')
circuit.populate_lane(2, qutip.tensor(rho1, rho2), 'rho')

#circuit.add_gate(MeasureGate([1], name='m2'))
#circuit.add_gate(MasterEquationGate([1], name='meq'))
circuit.add_gate(SecondTimeEvolutionGate([2,1], name='U2'))
circuit.add_gate(MeasureGate([1], name='ms'), 0)
circuit.add_gate(FirstTimeEvolutionGate([2,1], name='U1'))
circuit.add_gate(DephasingGate([1], name='u1', control_lane=0))
circuit.add_gate(MeasureGate([1], name='mi'), 0)
#circuit.draw()

for t in range(10):
    print(f"T = {t}")
    # Inject a new Ancilla in the circuit
    circuit.populate_lane(1, eta, 'eta')
    if t == 1:
        circuit.remove_gate(0)
    if t > 3:
        # Switch the phase controller
        circuit.update_lane(0, system=1, name=f'c{t}')
    if t == 5:
        circuit.add_gate(MeasureGate([1], name='mf'), position=2)
    circuit.draw()
    circuit.run()

Created U2 gate with code 378
Created ms gate with code 967
Created U1 gate with code 267
Created u1 gate with code 626
Created mi gate with code 892
T = 0
          1    2    3    4    5  
c1  ---- ----   x  ---- ---- ---- ---
eta ----  mi   u1   U1   ms   U2  ---
rho ---- ---- ----  U1  ----  U2  ---



MI measure

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.5 +0.j   0.  +0.j   0.  +0.j  ]
 [0.  +0.j   0.25+0.j   0.  +0.25j]
 [0.  +0.j   0.  -0.25j 0.25+0.j  ]]

(75, 75)
[[5, 5, 3], [5, 5, 3]]
[[5, 5, 3], [5, 5, 3]]


MS measure

Quantum object: dims = [[5, 5, 3], [5, 5, 3]], shape = (75, 75), type = oper, isherm = True
Qobj data =
[[0.49750416+0.j   0.        +0.j   0.        +0.j   ... 0.        +0.j
  0.        +0.j   0.        +0.j  ]
 [0.        +0.j   0.25      +0.j   0.        +0.25j ... 0.        +0.j
  0.        +0.j   0.        +0.j  ]
 [0.        +0.j   0.        -0.25j 0.25      +0.j   ... 0.        +0.j
  0.        +0.j   0.        +0.j  ]
 ...
 [0.        +0.j   0.        +0.j   0.        +0.j   ... 0.        +0.j
  0.        +0.j   0.        +0.j  ]
 [0.        +0.j   0.        +0.j   0.        +0.j   ... 0.        +0.j
  0.        +0.j   0.        +0.j  ]
 [0.        +0.j   0.        +0.j   0.        +0.j   ... 0.        +0.j
  0.        +0.j   0.        +0.j  ]]

(75, 75)
[[5, 5, 3], [5, 5, 3]]
[[5, 5, 3, 5, 5, 3], [5, 5, 3, 5, 5, 3]]


TypeError: Incompatible Qobj shapes